# LLaMA-Factory-HPC
https://github.com/hiyouga/LLaMA-Factory

In [ ]:
# 初始環境設定
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path
current_foldr=!pwd
current_foldr=current_foldr[0]
current_foldr

### 安裝 library

In [ ]:
!git clone https://github.com/hiyouga/LLaMA-Factory.git

In [ ]:
%cd LLaMA-Factory

In [ ]:
!pip install -r requirements.txt -q

In [ ]:
!pip install bitsandbytes kaleido cohere openai -q

In [ ]:
%%bash
#METHOD 05 ds_config.json
cat << EOF >  src/train_web_demo.py
from llmtuner import create_ui


def main():
    demo = create_ui()
    demo.queue()
    demo.launch(server_port=9000, server_name="$(hostname -s)", share=False, inbrowser=True)


if __name__ == "__main__":
    main()
EOF


In [ ]:
%%bash
node_hostname=$(hostname -s)
node_ip=$(cat /etc/hosts |grep "$(hostname -a)" | awk '{print $1}')
# PORT
#noed_port_genai=$(python -c "import socket; s = socket.socket(socket.AF_INET, socket.SOCK_STREAM); s.bind(('', 0)); addr = s.getsockname(); s.close(); print(addr[1])")
noed_port_genai=9000

# SSH FORWARDING
ssh_cmd="ssh -L ${noed_port_genai}:${node_hostname}:${noed_port_genai} $(whoami)@t3-c4.nchc.org.tw"
echo "SSH:"
echo ${ssh_cmd}
echo ""
echo "URL: http://localhost:${noed_port_genai}"
echo ""
 
CUDA_VISIBLE_DEVICES=0 HF_TOKEN='hf_xxxxxxxxxxxxxxxxxxxx' python src/train_web_demo.py

In [ ]:
# 強制刪除服務
!ps -ef |grep train_web | awk '{print $2}' | xargs kill -9